In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random

import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

%matplotlib inline

# фиксируем все seed
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [2]:
# параметры распределений
mu0, sigma0 = -2., 1.
mu1, sigma1 = 3., 2.

In [3]:
# функция для генерации выборки
def sample(d0, d1, mu0, mu1, sigma0, sigma1, n=32):
    # генерация одномерного случая
    x0 = d0.sample((n,)).numpy().reshape(-1)
    x1 = d1.sample((n,)).numpy().reshape(-1)
    # вычисляем матрицу корреляций
    r = np.corrcoef(x0,x1)
    # Т.К. r_xy = r_yx, то необходимый коэффициент корреляции равен 
    r = r[0][1]
    # Зададим многомерное распределение
    # матрица средних значений 
    mu = torch.tensor([mu0,mu1],dtype=torch.float)
    # матрица ковариацией
    cov_mat = torch.tensor([[sigma0**2, r*sigma0*sigma1],[r*sigma0*sigma1, sigma1**2]],dtype=torch.float)
    #многомерное распределение
    dd0 = torch.distributions.multivariate_normal.MultivariateNormal(mu, cov_mat)
    # сэмплируем из многомерного распределения
    x = dd0.sample((n,))
    # обозначим за 1 наблюдения которые больше своих средних в распределении
    y = []
    for x_ in x: 
        if x_[0]>= mu0 and x_[1]>= mu1:
            y.append(1)
        else : 
            y.append(0)
    y_tensor = torch.tensor(y,dtype = torch.float).reshape(-1,1)
    return x,y_tensor

In [4]:
# параметры для генерации нормального распределения
d0 = torch.distributions.Normal(torch.tensor([mu0]), torch.tensor([sigma0]))
d1 = torch.distributions.Normal(torch.tensor([mu1]), torch.tensor([sigma1]))

In [5]:
x,y = sample(d0, d1, mu0, mu1, sigma0, sigma1)

In [6]:
# на вход подается два распределения вместо одного, поэтому nn.Linear необходимо изменить параметр
layer = nn.Linear(2, 1)
print([p.data[0] for p in layer.parameters()])
layer_opt = optim.SGD(lr=1e-3, params=list(layer.parameters()))

[tensor([-0.2474, -0.5796]), tensor(-0.1504)]


In [7]:
log_freq = 500
for i in range(10000):
    if i%log_freq == 0:
        with torch.no_grad():
            x, y = sample(d0, d1,mu0, mu1, sigma0, sigma1, 100000)
            out = torch.sigmoid(layer(x))
            loss = F.binary_cross_entropy(out, y)
        print('Ошибка после %d итераций: %f' %(i/log_freq, loss))
    layer_opt.zero_grad()
    x, y = sample(d0, d1, mu0, mu1, sigma0, sigma1,1024)
    out = torch.sigmoid(layer(x))
    loss = F.binary_cross_entropy(out, y)
    loss.backward()
    layer_opt.step()

Ошибка после 0 итераций: 0.960796
Ошибка после 1 итераций: 0.738502
Ошибка после 2 итераций: 0.612603
Ошибка после 3 итераций: 0.533877
Ошибка после 4 итераций: 0.483612
Ошибка после 5 итераций: 0.446588
Ошибка после 6 итераций: 0.419595
Ошибка после 7 итераций: 0.396793
Ошибка после 8 итераций: 0.381613
Ошибка после 9 итераций: 0.368812
Ошибка после 10 итераций: 0.356868
Ошибка после 11 итераций: 0.346486
Ошибка после 12 итераций: 0.339361
Ошибка после 13 итераций: 0.334149
Ошибка после 14 итераций: 0.328734
Ошибка после 15 итераций: 0.322655
Ошибка после 16 итераций: 0.318639
Ошибка после 17 итераций: 0.314679
Ошибка после 18 итераций: 0.311106
Ошибка после 19 итераций: 0.308451
